In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split

# 1. 교통량 데이터 로드
weekdays_data = pd.read_excel('../data/weekday_traffic.xlsx') 

weekdays_data.head()

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,17시,18시,19시,20시,21시,22시,23시,혼잡,평균교통량,새로운 혼잡
0,20240801,목,마포대교,C-07,유입,마포대교남단->마포대교북단,1121.100127,739.396817,603.646236,429.588089,...,3982.454958,3465.886595,2569.941716,2727.605192,2393.113295,2159.360088,1520.158999,True,2564.041667,False
1,20240802,금,마포대교,C-07,유입,마포대교남단->마포대교북단,1064.668712,758.031761,487.068186,397.855528,...,3890.244416,3425.769366,2954.749926,2613.879546,2301.021973,2244.874364,1496.910955,True,2516.208333,False
2,20240805,월,마포대교,C-07,유입,마포대교남단->마포대교북단,616.797360,464.364338,337.000433,282.774209,...,3865.243889,3743.189576,2772.238162,2506.984733,2199.044578,2080.754404,1571.072355,False,2355.000000,False
3,20240806,화,마포대교,C-07,유입,마포대교남단->마포대교북단,1063.667580,687.378212,416.206495,339.725827,...,3839.879003,4042.135076,2663.051395,2346.262683,2647.551173,1916.299569,1523.996405,False,2488.666667,False
4,20240807,수,마포대교,C-07,유입,마포대교남단->마포대교북단,1042.007691,754.815160,531.468487,363.953378,...,3840.413752,3650.694194,2892.151038,2472.565715,2729.293114,2335.362316,1478.987119,True,2549.875000,False


In [7]:
# 2. 독립 변수: 각 날의 8시만 사용 (날짜별로 동일한 시간)
# np.arange()는 연속된 수의 배열을 생성하며, reshape()은 차원을 변환 (참고: https://numpy.org/doc/stable/reference/generated/numpy.arange.html)
days = np.arange(len(weekdays_data)).reshape(-1, 1)[:-1]  # 각 날을 나타내는 인덱스를 독립 변수로 사용
days

array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19],
       [20],
       [21],
       [22],
       [23],
       [24],
       [25],
       [26],
       [27],
       [28],
       [29],
       [30],
       [31],
       [32],
       [33],
       [34],
       [35],
       [36],
       [37],
       [38],
       [39],
       [40]])

In [8]:
traffic_at_8am = weekdays_data.loc[:, '8시'].values[:-1]   # 열 이름이 '8시'인 교통량 데이터를 종속 변수로 사용

# 4. 데이터 분리 (훈련 데이터 80%, 테스트 데이터 20%)
# train_test_split은 데이터를 훈련과 테스트 데이터로 분리하는 함수 (참고: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
# random_state=42로 고정
X_train, X_test, y_train, y_test = train_test_split(days, traffic_at_8am, test_size=0.2, random_state=42)

# 5. 선형 회귀 모델 생성 및 학습 (훈련 데이터로 학습)
# LinearRegression 객체를 생성하여 fit() 함수로 학습 (참고: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [9]:
# 마지막 5일의 인덱스 (예: 데이터가 31일이면, 인덱스 26~30)
indices = np.arange(25, 31).reshape(-1,1)
predicted_traffic = model.predict(indices)
print(predicted_traffic)

[3692.61228324 3679.48605374 3666.35982423 3653.23359472 3640.10736522
 3626.98113571]


In [10]:
# 6. 마지막 날의 8시 교통량 예측
# 모델의 predict() 함수를 사용하여 예측값을 도출 (참고: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.predict)
predicted_traffic = model.predict(np.array([[len(weekdays_data) -1]]))  # 마지막 날의 인덱스
predicted_traffic

array([3482.59261114])

In [11]:

# 7. 마지막 날의 8시 실제 교통량 가져오기
# loc[]을 사용하여 마지막 날의 데이터를 가져옴 (참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html)
last_day_traffic = weekdays_data.loc[weekdays_data.index[-1], '8시']

# 8. 예측된 값과 실제 값을 출력
print(f"실제 Traffic: {last_day_traffic:.2f}, 예측한 Traffic: {predicted_traffic[0]:.2f}")

실제 Traffic: 3211.69, 예측한 Traffic: 3482.59
